# NML import notebook
This notebook downloads data from No More Leaks and import it into the NML database

# Set constants

In [ ]:
# Change!
DATE="2024-09-11"
# Keep the same
DATA_DIR="../NML"
SHARED_DIR="../SHARED_DBS"
OUT_DB=f"{SHARED_DIR}/NML.sqlite3"

# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

# Import data

In [ ]:
!ls $DATA_DIR


In [ ]:
md5_files = glob(f"{DATA_DIR}/*.md5")

In [ ]:
md5_files

# Open DB (or create it)

In [ ]:
# Open database
conn = sqlite3.connect(OUT_DB)
cur = conn.cursor()

In [ ]:
conn

# Check if date already in database

In [ ]:
cur.execute(f"SELECT nml_hash FROM hashes WHERE last = '{DATE}' LIMIT 1;")
row = cur.fetchone()
if row :
    print(f"DATE is set to '{DATE}', but there are rows in the database with this date.", file=sys.stderr)
else:
    print("OK")

# Import records

In [ ]:
count=0
for md5_file in md5_files :
    md5fh = open(md5_file, "r")
    md5 = ""
    for line in md5fh.readlines() :
        if md5 == "" :
            md5 = line.split()[0]
            file = line.split()[1]
        cur.execute("SELECT count(*) FROM files WHERE md5 = ?;",(md5,))
        row = cur.fetchone()
    if row[0] == 0 :
        # md5 is not yet in database, let's import these hashes
        print(f"Importing '{file}'")
        gzfh = gzip.open(f"{DATA_DIR}/{file}", 'rt')
        for hashln in gzfh:
            hashln = hashln.strip()
            cur.execute("INSERT OR IGNORE INTO hashes ( nml_hash ) values ( ? );" , ( hashln, ) )
            cur.execute("UPDATE hashes SET last = ? WHERE nml_hash = ?;" , ( DATE, hashln ) )
            count = count + 1
            if count % 10_000 == 0 :
                print(f"{count:,}  ", end="\r")
            if count % 1_000_000 == 0 :
                conn.commit()
        conn.commit()
        # Set first for hashes that are imported first
        cur.execute("UPDATE hashes SET first = last WHERE first IS NULL;");
        conn.commit()
        # Mark file as imported
        cur.execute("INSERT INTO files ( name, md5 ) values ( ?, ? );",(file, md5))
        conn.commit()
        print(f"{count:,}  ")
    else:
        print(f"Skipping {md5_file[7:-4]}") 
    conn.commit()
    

In [ ]:
# Commit database to release lock (if step above has failed)
conn.commit()